# Optimized Stacking Ensemble

## 1. Objective
Use the **Best Hyperparameters** found by Optuna to train our strongest possible Stacking Ensemble.
This aims to beat the current baseline of **23.97**.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
import sys
import os

# Add src to path
sys.path.append(os.path.abspath('..'))

from src.models import XGBoostModel, LightGBMModel, CatBoostModel, StackingEnsemble

# Load Data
train_df = pd.read_csv('../data/processed/train_featurized.csv')
test_df = pd.read_csv('../data/processed/test_featurized.csv')

X = train_df.drop(['id', 'SMILES', 'Tm'], axis=1)
y = train_df['Tm']
X_test = test_df.drop(['id', 'SMILES'], axis=1)

# Fill NaNs for safety
X = X.fillna(X.mean())
X_test = X_test.fillna(X.mean())

## 2. Insert Best Parameters
**Paste the output from `11_optuna_tuning.ipynb` here.**

In [2]:
# --- PASTE OPTUNA RESULTS BELOW ---

# Example placeholders (REPLACE THESE with your actual results)
best_xgb_params = {
    'n_estimators': 3000,
    'learning_rate': 0.01,  # Likely found ~0.01-0.05
    'max_depth': 6,         # Likely 4-8
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_jobs': -1,
    'random_state': 42
}

best_lgb_params = {
    'n_estimators': 3000,
    'learning_rate': 0.01,
    'num_leaves': 31,       # Likely 20-60
    'min_child_samples': 20,
    'n_jobs': -1,
    'random_state': 42,
    'verbosity': -1
}

best_cat_params = {
    'iterations': 3000,
    'learning_rate': 0.01,
    'depth': 6,             # Likely 4-8
    'l2_leaf_reg': 3.0,
    'loss_function': 'MAE',
    'verbose': False,
    'random_state': 42,
    'task_type': 'CPU'
}

# ----------------------------------

## 3. Train Optimized Stacking Ensemble

In [3]:
base_models = {
    'XGBoost': XGBoostModel(best_xgb_params),
    'LightGBM': LightGBMModel(best_lgb_params),
    'CatBoost': CatBoostModel(best_cat_params)
    # We omit NeuralNet here to keep it pure Tree-based (usually sharper for tabular)
    # But you can uncomment it if you want to include the default NN
    # 'NeuralNet': NeuralNetworkModel({'epochs': 100, 'batch_size': 32})
}

print("Training Optimized Stacking Ensemble...")
stacking_model = StackingEnsemble(
    base_models=base_models,
    meta_model=Ridge(alpha=0.5),
    n_folds=10  # Increased to 10-Fold for maximum robustness
)

stacking_model.fit(X, y)

print("Predicting on Test Set...")
final_preds = stacking_model.predict(X_test)

# Save Submission
sub = pd.DataFrame({'id': test_df['id'], 'Tm': final_preds})
sub.to_csv('../submissions/submission_optuna_stacking.csv', index=False)
print("Saved optimized submission to submissions/submission_optuna_stacking.csv")

Training Optimized Stacking Ensemble...
Generating OOF predictions with 10 folds...
  Processing XGBoost...
  Processing LightGBM...
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[2992]	valid_0's l1: 28.3207
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[2984]	valid_0's l1: 27.9078
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1940]	valid_0's l1: 28.1983
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2428]	valid_0's l1: 25.984
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[2023]	valid_0's l1: 26.1716
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1450]	valid_0's l1: 27.3206
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration 